Modifying the `calculate_cash_flows` method to generate the whole schedule from the `issue_date` to the `maturity_date` and then consider only values for dates posterior to the `ref_date`, taking into account a fraction of the coupon for the payment date next to the `ref_date`

In [41]:
import pandas as pd
import pandas as pd
import numpy as np
from scipy.optimize import brentq
from datetime import datetime

In [26]:
class Bondv0:
    def __init__(self, isin, issue_date, maturity_date,face_value=100, price=None, 
                 yield_to_maturity=None, coupon_rate=None, frequency=None, 
                 coupon_series=None, day_count_convention='30/360'):
        self.isin = isin
        self.face_value = face_value
        self.issue_date = pd.to_datetime(issue_date)
        self.maturity_date = pd.to_datetime(maturity_date)
        self.price = price
        self.yield_to_maturity = yield_to_maturity
        self.coupon_rate = coupon_rate
        self.frequency = frequency
        self.coupon_series = coupon_series
        self.day_count_convention = day_count_convention

    def calculate_days(self, date1, date2):
        if self.day_count_convention == 'Actual/Actual':
            return (date2 - date1).days
        elif self.day_count_convention == '30/360':
            return 360 * (date2.year - date1.year) + 30 * (date2.month - date1.month) + min(30, date2.day) - min(30, date1.day)

    def calculate_cash_flows(self, ref_date):
        cash_flows = pd.Series([self.coupon_rate * self.face_value] * ((self.maturity_date.year - self.issue_date.year) + 1), 
                               index=pd.date_range(start=self.issue_date, end=self.maturity_date, freq=pd.DateOffset(years=1))
                              )
        cash_flows.iloc[-1] += self.face_value  # Add face value to the last payment

        if ref_date < self.issue_date:
            return 0
        elif ref_date > self.maturity_date:
            return self.face_value
        else:
            for date in cash_flows.index:
                if date <= ref_date:
                    cash_flows.loc[date] = 0
                elif date == ref_date + pd.DateOffset(years=1):
                    days_in_year = (date - ref_date).days
                    cash_flows.loc[date] = self.coupon_rate * self.face_value * (days_in_year / 365)
            return cash_flows

    def calculate_price(self, yield_to_maturity, ref_date):
        cash_flows = self.calculate_cash_flows(ref_date)
        return np.sum([cf / (1 + yield_to_maturity)**(self.calculate_days(ref_date, date) / 365) for date, cf in cash_flows.items()])

    def calculate_yield_to_maturity(self, ref_date):
        def calculate_present_value(rate):
            return np.sum([cf / (1 + rate)**(self.calculate_days(ref_date, date) / 365) for date, cf in cash_flows.items()]) - self.price

        cash_flows = self.calculate_cash_flows(ref_date)
        return brentq(calculate_present_value, -1, 1)


In this modification:
- The `calculate_cash_flows` method generates the whole schedule from the `issue_date` to the `maturity_date` and then adjusts the cash flows based on the `ref_date`.
- It sets the cash flows to 0 for dates prior to the `ref_date`, calculates a fraction of the coupon for the payment date next to the `ref_date`, and keeps the original cash flows for dates after the `ref_date`.

These changes ensure that the cash flows are correctly adjusted based on the reference date provided for the bond calculations.

In [27]:
class Bondv1:
    def __init__(self, isin, face_value, issue_date, maturity_date, price=None, 
                 yield_to_maturity=None, coupon_rate=None, frequency=1, 
                 coupon_series=None, day_count_convention='30/360'):
        self.isin = isin
        self.face_value = face_value
        self.issue_date = pd.to_datetime(issue_date)
        self.maturity_date = pd.to_datetime(maturity_date)
        self.price = price
        self.yield_to_maturity = yield_to_maturity
        self.coupon_rate = coupon_rate
        self.frequency = frequency
        self.coupon_series = coupon_series
        self.day_count_convention = day_count_convention

    def calculate_days(self, date1, date2):
        if self.day_count_convention == 'Actual/Actual':
            return (date2 - date1).days
        elif self.day_count_convention == '30/360':
            return 360 * (date2.year - date1.year) + 30 * (date2.month - date1.month) + min(30, date2.day) - min(30, date1.day)

    def calculate_cash_flows(self, ref_date):
        cash_flows = pd.Series([self.coupon_rate * self.face_value] * ((self.maturity_date.year - self.issue_date.year) + 1), 
                               index=pd.date_range(start=self.issue_date, end=self.maturity_date, 
                                                   freq=pd.DateOffset(years=1))
                              )
        cash_flows.iloc[-1] += self.face_value  # Add face value to the last payment

        if ref_date < self.issue_date:
            return 0
        elif ref_date > self.maturity_date:
            return self.face_value
        else:
            for date in cash_flows.index:
                if date <= ref_date:
                    cash_flows.loc[date] = 0
                elif date == ref_date + pd.DateOffset(years=1/self.frequency):
                    days_in_year = (date - ref_date).days
                    cash_flows.loc[date] = self.coupon_rate * self.face_value * (days_in_year / (365/self.frequency))
            return cash_flows

    def calculate_price(self, yield_to_maturity, ref_date):
        cash_flows = self.calculate_cash_flows(ref_date)
        return np.sum([cf / (1 + yield_to_maturity)**(self.calculate_days(ref_date, date) / 365) for date, cf in cash_flows.items()])

    def calculate_yield_to_maturity(self, ref_date):
        def calculate_present_value(rate):
            return np.sum([cf / (1 + rate)**(self.calculate_days(ref_date, date) / 365) for date, cf in cash_flows.items()]) - self.price

        cash_flows = self.calculate_cash_flows(ref_date)
        return brentq(calculate_present_value, 0.0001, 1)


In [33]:
class Bondv2:
    def __init__(self, isin, face_value, issue_date, maturity_date, price=None, yield_to_maturity=None, 
                 coupon_rate=None, frequency=1, coupon_series=None, day_count_convention='30/360'):
        self.isin = isin
        self.face_value = face_value
        self.issue_date = pd.to_datetime(issue_date)
        self.maturity_date = pd.to_datetime(maturity_date)
        self.price = price
        self.yield_to_maturity = yield_to_maturity
        self.coupon_rate = coupon_rate
        self.frequency = frequency
        self.coupon_series = coupon_series
        self.day_count_convention = day_count_convention

    def calculate_days(self, date1, date2):
        if self.day_count_convention == 'Actual/Actual':
            return (date2 - date1).days
        elif self.day_count_convention == '30/360':
            return 360 * (date2.year - date1.year) + 30 * (date2.month - date1.month) + min(30, date2.day) \
                    - min(30, date1.day)

    def calculate_cash_flows(self, ref_date):
        num_periods = (self.maturity_date.year - self.issue_date.year) * self.frequency
        cash_flows = pd.Series([self.coupon_rate * self.face_value / self.frequency] * num_periods, 
                               index=pd.date_range(start=self.issue_date, periods=num_periods, 
                                                   freq=pd.DateOffset(months=12/self.frequency))
                              )
        cash_flows.iloc[-1] += self.face_value  # Add face value to the last payment

        if ref_date < self.issue_date:
            return 0
        elif ref_date > self.maturity_date:
            return self.face_value
        else:
            for date in cash_flows.index:
                if date <= ref_date:
                    cash_flows.loc[date] = 0
                elif date <= ref_date + pd.DateOffset(months=12/self.frequency):
                    days_in_period = self.calculate_days(ref_date, date)
                    cash_flows.loc[date] = self.coupon_rate * self.face_value \
                    * (days_in_period / 365)
            return cash_flows

    def calculate_price(self, yield_to_maturity, ref_date):
        cash_flows = self.calculate_cash_flows(ref_date)
        return np.sum([cf / (1 + yield_to_maturity)**(self.calculate_days(ref_date, date) / 365) for date, 
                       cf in cash_flows.items()])

    def calculate_yield_to_maturity(self, ref_date):
        def calculate_present_value(rate):
            return np.sum([cf / (1 + rate)**(self.calculate_days(ref_date, date) / 365) for date, 
                           cf in cash_flows.items()]) - self.price

        cash_flows = self.calculate_cash_flows(ref_date)
        return brentq(calculate_present_value, -0.5, 1)


In [36]:
class Bond:
    def __init__(self, isin, face_value, issue_date, maturity_date, price=None, yield_to_maturity=None, 
                 coupon_rate=None, frequency=1, coupon_series=None, day_count_convention='30/360'):
        self.isin = isin
        self.face_value = face_value
        self.issue_date = pd.to_datetime(issue_date)
        self.maturity_date = pd.to_datetime(maturity_date)
        self.price = price
        self.yield_to_maturity = yield_to_maturity
        self.coupon_rate = coupon_rate
        self.frequency = frequency
        self.coupon_series = coupon_series
        self.day_count_convention = day_count_convention

    def calculate_days(self, date1, date2):
        if self.day_count_convention == 'Actual/Actual':
            return (date2 - date1).days
        elif self.day_count_convention == '30/360':
            return 360 * (date2.year - date1.year) + 30 * (date2.month - date1.month) + min(30, date2.day) \
                    - min(30, date1.day)
        elif self.day_count_convention == 'Actual/360':
            return (date2 - date1).days
        elif self.day_count_convention == 'Actual/365':
            return (date2 - date1).days

    def calculate_fraction(self, date1, date2):
        if self.day_count_convention == 'Actual/Actual':
            return self.calculate_days(date1, date2) / 365
        elif self.day_count_convention == '30/360':
            return self.calculate_days(date1, date2) / 360
        elif self.day_count_convention == 'Actual/360':
            return self.calculate_days(date1, date2) / 360
        elif self.day_count_convention == 'Actual/365':
            return self.calculate_days(date1, date2) / 365

    def calculate_cash_flows(self, ref_date):
        num_periods = (self.maturity_date.year - self.issue_date.year) * self.frequency
        cash_flows = pd.Series([self.coupon_rate * self.face_value / self.frequency] * num_periods, 
                               index=pd.date_range(start=self.issue_date, periods=num_periods, 
                               freq=pd.DateOffset(months=12/self.frequency))
                              )
        cash_flows.iloc[-1] += self.face_value  # Add face value to the last payment

        if ref_date < self.issue_date:
            return 0
        elif ref_date > self.maturity_date:
            return self.face_value
        else:
            for date in cash_flows.index:
                if date <= ref_date:
                    cash_flows.loc[date] = 0
                elif date <= ref_date + pd.DateOffset(months=12/self.frequency):
                    days_in_period = self.calculate_days(ref_date, date)
                    cash_flows.loc[date] = self.coupon_rate * self.face_value \
                    * self.calculate_fraction(ref_date, date)
            return cash_flows

    def calculate_price(self, yield_to_maturity, ref_date):
        cash_flows = self.calculate_cash_flows(ref_date)
        return np.sum([cf / (1 + yield_to_maturity)**(self.calculate_fraction(ref_date, date)) \
                       for date, cf in cash_flows.items()])

    def calculate_yield_to_maturity(self, ref_date):
        def calculate_present_value(rate):
            return np.sum([cf / (1 + rate)**(self.calculate_fraction(ref_date, date)) \
                           for date, cf in cash_flows.items()]) - self.price

        cash_flows = self.calculate_cash_flows(ref_date)
        return brentq(calculate_present_value, -0.7, 1)

    def calculate_duration(self, ref_date):
        cash_flows = self.calculate_cash_flows(ref_date)
        durations = [self.calculate_fraction(ref_date, date) * cf * self.calculate_fraction(ref_date, date)\
                     / (1 + self.calculate_yield_to_maturity(ref_date))**self.calculate_fraction(ref_date, date) for date, cf in cash_flows.items()]
        return np.sum(durations)

    def calculate_modified_duration(self, ref_date):
        price = self.price(ref_date)
        yield_to_maturity = self.calculate_yield_to_maturity(ref_date)
        duration = self.calculate_duration(ref_date)
        return duration / (1 + yield_to_maturity)


In this modification:
- The `calculate_duration` method computes the duration of the bond for a given reference date by summing the present value weighted time to each cash flow.
- The `calculate_modified_duration` method calculates the modified duration of the bond at a given reference date by dividing the duration by 1 plus the yield to maturity at that date.
- These additions allow for computing both the duration and modified duration of the bond for a specific reference date.

In [37]:
# Create an instance of the Bond class with a qsemi-annual payment frequency
bond_semester = Bond(isin='US123456789', face_value=100, issue_date='2022-01-01', maturity_date='2027-01-01',
                      price = 109, coupon_rate=0.05, frequency=2)

# Reference date equal to the issue date
ref_date_equal_issue = pd.to_datetime('2022-01-01')

# Calculate cash flows for the reference date equal to the issue date
cash_flows_equal_issue = bond_semester.calculate_cash_flows(ref_date_equal_issue)
print("Cash Flows (Reference Date = Issue Date):")
print(cash_flows_equal_issue)

# Reference date between issue date and maturity date
ref_date_between = pd.to_datetime('2024-10-05')

# Calculate cash flows for the reference date between issue date and maturity date
cash_flows_between = bond_semester.calculate_cash_flows(ref_date_between)
print("\nCash Flows (Reference Date = 2024-10-05):")
print(cash_flows_between)

yield_to_maturity = bond_semester.calculate_yield_to_maturity(ref_date_between)
print("\nYield to Maturity of the bond (Reference Date = 2024-10-05):")
print(yield_to_maturity)

Cash Flows (Reference Date = Issue Date):
2022-01-01      0.0
2022-07-01      2.5
2023-01-01      2.5
2023-07-01      2.5
2024-01-01      2.5
2024-07-01      2.5
2025-01-01      2.5
2025-07-01      2.5
2026-01-01      2.5
2026-07-01    102.5
Freq: <DateOffset: months=6.0>, dtype: float64

Cash Flows (Reference Date = 2024-10-05):
2022-01-01      0.000000
2022-07-01      0.000000
2023-01-01      0.000000
2023-07-01      0.000000
2024-01-01      0.000000
2024-07-01      0.000000
2025-01-01      1.194444
2025-07-01      2.500000
2026-01-01      2.500000
2026-07-01    102.500000
Freq: <DateOffset: months=6.0>, dtype: float64

Yield to Maturity of the bond (Reference Date = 2024-10-05):
-0.0016617020658115829


To add a `Portfolio` class that consists of a portfolio of bond objects with quantities for each bond 
as integers, and to include methods for calculating price, yield to maturity, duration, 
and modified duration in the `Portfolio` class :

In [38]:
class Portfoliov0:
    def __init__(self):
        self.bonds = {}

    def add_bond(self, bond, quantity):
        self.bonds[bond] = quantity

    def calculate_price(self, ref_date):
        total_price = sum(bond.price(ref_date) * quantity for bond, quantity in self.bonds.items())
        return total_price

    def calculate_yield_to_maturity(self, ref_date):
        total_value = sum(bond.price(ref_date) * quantity for bond, quantity in self.bonds.items())
        total_face_value = sum(bond.face_value * quantity for bond, quantity in self.bonds.items())
        return total_value / total_face_value

    def calculate_duration(self, ref_date):
        total_duration = sum(bond.calculate_duration(ref_date) * quantity for bond, quantity in self.bonds.items())
        return total_duration

    def calculate_modified_duration(self, ref_date):
        total_modified_duration = sum(bond.calculate_modified_duration(ref_date) * quantity for bond, quantity in self.bonds.items())
        return total_modified_duration

    @classmethod
    def from_dataframe(cls, df):
        portfolio = cls()
        for index, row in df.iterrows():
            bond = Bond(row['isin'], row['face_value'], row['issue_date'], row['maturity_date'], row['price'], row['yield_to_maturity'], row['coupon_rate'], row['frequency'], row['coupon_series'], row['day_count_convention'])
            portfolio.add_bond(bond, row['quantity'])
        return portfolio


In [40]:
class Portfolio:
    def __init__(self):
        self.bonds = {}

    def add_bond(self, bond, quantity):
        self.bonds[bond] = quantity

    def modify_bond(self, bond, new_quantity, **kwargs):
        if bond in self.bonds:
            self.bonds[bond] = new_quantity
            for key, value in kwargs.items():
                setattr(bond, key, value)
        else:
            print("Bond not found in the portfolio.")

    def modify(self, new_quantities):
        for bond, new_quantity in new_quantities.items():
            if bond in self.bonds:
                self.bonds[bond] = new_quantity
            else:
                print(f"{bond} not found in the portfolio.")

    def calculate_price(self, ref_date, yield_df=None):
        total_price = 0
        for bond, quantity in self.bonds.items():
            if yield_df is not None and bond.isin in yield_df.index:
                if bond.yield_to_maturity is not None:
                    print(f"For bond with ISIN {bond.isin},\
                    the yield to maturity was already instantiated to {bond.yield_to_maturity}")
                bond.yield_to_maturity = yield_df.loc[bond.isin, 'yield_to_maturity']

            total_price += bond.calculate_price(bond.yield_to_maturity, ref_date) * quantity

        return total_price

    def calculate_yield_to_maturity(self, ref_date):
        total_value = sum(bond.price(ref_date) * quantity for bond, quantity in self.bonds.items())
        total_face_value = sum(bond.face_value * quantity for bond, quantity in self.bonds.items())
        return total_value / total_face_value

    def calculate_duration(self, ref_date):
        total_duration = sum(bond.calculate_duration(ref_date) * quantity for bond, quantity in self.bonds.items())
        return total_duration

    def calculate_modified_duration(self, ref_date):
        total_modified_duration = sum(bond.calculate_modified_duration(ref_date) * quantity for bond, quantity in self.bonds.items())
        return total_modified_duration

    @classmethod
    def from_dataframe(cls, df):
        portfolio = cls()
        for index, row in df.iterrows():
            bond = Bond(row['isin'], row['face_value'], row['issue_date'], row['maturity_date'], row['price'], row['yield_to_maturity'], row['coupon_rate'], row['frequency'], row['coupon_series'], row['day_count_convention'])
            portfolio.add_bond(bond, row['quantity'])
        return portfolio


In [42]:
# Create a sample bond dataframe
data = {
    'isin': ['ISIN1', 'ISIN2'],
    'face_value': [1000, 1500],
    'issue_date': ['2022-01-01', '2021-01-01'],
    'maturity_date': ['2027-01-01', '2026-01-01'],
    'price': [950, 1400],
    'yield_to_maturity': [0.05, 0.04],
    'coupon_rate': [0.06, 0.05],
    'frequency': [2, 1],
    'coupon_series': [None, None],
    'day_count_convention': ['30/360', 'Actual/Actual'],
    'quantity': [10, 5]
}

df = pd.DataFrame(data)

# Instantiate the first Portfolio with explicit bonds
portfolio1 = Portfolio()
bond1 = Bond('ISIN1', 1000, '2022-01-01', '2027-01-01', 950, 0.05, 0.06, 2, None, '30/360')
bond2 = Bond('ISIN2', 1500, '2021-01-01', '2026-01-01', 1400, 0.04, 0.05, 1, None, 'Actual/Actual')
portfolio1.add_bond(bond1, 10)
portfolio1.add_bond(bond2, 5)

# Instantiate the second Portfolio from the DataFrame
portfolio2 = Portfolio.from_dataframe(df)

# Create a DataFrame with ISIN and yield_to_maturity values
yield_df = pd.DataFrame({'isin': ['ISIN1', 'ISIN2'], 'yield_to_maturity': [0.055, 0.045]})
yield_df.set_index('isin', inplace=True)

# Calculate the price of the two portfolios with yield_to_maturity from the DataFrame
price_portfolio1 = portfolio1.calculate_price(datetime.now(), yield_df)
price_portfolio2 = portfolio2.calculate_price(datetime.now(), yield_df)

print(f"Price of Portfolio 1: {price_portfolio1}")
print(f"Price of Portfolio 2: {price_portfolio2}")


For bond with ISIN ISIN1,                    the yield to maturity was already instantiated to 0.05
For bond with ISIN ISIN2,                    the yield to maturity was already instantiated to 0.04
For bond with ISIN ISIN1,                    the yield to maturity was already instantiated to 0.05
For bond with ISIN ISIN2,                    the yield to maturity was already instantiated to 0.04
Price of Portfolio 1: 10281.313915531164
Price of Portfolio 2: 10281.313915531164


- We first create a sample bond DataFrame.
- We instantiate the first portfolio (`portfolio1`) with explicit bonds using the `add_bond` method.
- We instantiate the second portfolio (`portfolio2`) from the DataFrame using the `from_dataframe` method.
- We create a DataFrame (`yield_df`) with ISIN and `yield_to_maturity` values.
- We calculate the price of the two portfolios (`portfolio1` and `portfolio2`) using the `calculate_price` method 
  with the `yield_to_maturity` values from the `yield_df` DataFrame.
